In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

import yfinance as yf

import os

from arch import arch_model

In [2]:
# Download data automatically if file not present
name_bhp = 'bhp_yf_2000_2021.csv'

# BHP
if not os.path.isfile(name_bhp):
    data_bhp = yf.download("BHP.AX", start='2000-01-01', end='2021-07-20')
    data_bhp.to_csv(name_bhp)
    
data = pd.read_csv(name_bhp, index_col='Date', parse_dates=True, dayfirst=True)

In [3]:
# It is actually not necessary to truncate the return
# series; we do it to speed up the forecasting later.
p = data['Adj Close']
r_all = 100 * np.log(p).diff().dropna()
r = r_all[:'2017-10-16']

In [4]:
garch_t = arch_model(r, mean='Constant', vol='GARCH', p=1, q=1, dist='StudentsT')
garch_t_fit = garch_t.fit( disp= 'off' )

gjr_t = arch_model(r, mean='Constant', vol='GARCH', p=1, o=1, q=1, dist='StudentsT')
gjr_t_fit = gjr_t.fit( disp= 'off' )

egarch_n = arch_model(r, mean='Constant', vol='EGARCH', p=1, o=1, q=1, dist='normal')
egarch_n_fit = egarch_n.fit( disp= 'off' )

egarch_t = arch_model(r, mean='Constant', vol='EGARCH', p=1, o=1, q=1, dist='StudentsT')
egarch_t_fit = egarch_t.fit( disp= 'off' )

In [5]:
h = len(r_all['2017-10-17':])
f_garch_t = garch_t_fit.forecast(
    horizon=h, align='origin', reindex=False).variance.values.T

### GARCH(1, 1)-t
$$\sigma_t^2 = 0.0246 + 0.0541a_{t-1}^2 + 0.9391\sigma_{t-1}^2$$

$$\mathbb{E}[\sigma_t^2] = 0.0246 + (0.0541 + 0.9391)\mathbb{E}[\sigma_{t-1}^2]$$

In [6]:
print(garch_t_fit.summary())

                        Constant Mean - GARCH Model Results                         
Dep. Variable:                    Adj Close   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:               -8816.38
Distribution:      Standardized Student's t   AIC:                           17642.8
Method:                  Maximum Likelihood   BIC:                           17674.8
                                              No. Observations:                 4500
Date:                      Fri, Oct 29 2021   Df Residuals:                     4499
Time:                              11:09:31   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu        

In [35]:
a0 = garch_t_fit.params['omega']
a1 = garch_t_fit.params['alpha[1]']
b1 = garch_t_fit.params['beta[1]']

In [36]:
val = {}

val["a_{t-1}"] = garch_t_fit.resid[-1]
val["σ_{t-1}"] = garch_t_fit.conditional_volatility[-1]

In [37]:
val

{'a_{t-1}': 2.1495878884992825, 'σ_{t-1}': 1.1158072006467992}

In [38]:
# First prediction if the previous shock and std are known
a0 + a1 * (val["a_{t-1}"] ** 2) + b1 * (val["σ_{t-1}"] ** 2)

1.4436536035059597

In [39]:
a0 + a1 * (2.1496 ** 2) + b1 * (1.1158**2)

1.4436413287851022

In [44]:
var = 1.5446255943779397
a0 + a1 * var + b1 * var

1.5586598124861721

In [41]:
f_garch_t[:10].flatten()

array([1.4436536 , 1.45837564, 1.47299739, 1.48751954, 1.50194276,
       1.51626773, 1.53049512, 1.5446256 , 1.55865981, 1.57259843])



### GJR(1)-GARCH(1, 1)-t
$$\sigma_t^2 = 0.0263 + 0.0353a_{t-1}^2 + (0.9404 + 0.0327I_{t-1})\sigma_{t-1}^2$$

$$\mathbb{E}[\sigma_t^2] = 0.0263 + (0.0353 + 0.9404 + 0.0327 \times 0.5)\mathbb{E}[\sigma_{t-1}^2]$$

In [21]:
print(gjr_t_fit.summary())

                      Constant Mean - GJR-GARCH Model Results                       
Dep. Variable:                    Adj Close   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                        GJR-GARCH   Log-Likelihood:               -8811.00
Distribution:      Standardized Student's t   AIC:                           17634.0
Method:                  Maximum Likelihood   BIC:                           17672.5
                                              No. Observations:                 4500
Date:                      Fri, Oct 29 2021   Df Residuals:                     4499
Time:                              11:09:31   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu  

In [22]:
f_gjr_t = gjr_t_fit.forecast(
    horizon=h, align='origin', reindex=False).variance.values.T

In [25]:
# GJR-GARCH(1,1)
a0 = gjr_t_fit.params['omega']
a1 = gjr_t_fit.params['alpha[1]']
b1 = gjr_t_fit.params['beta[1]']
g1 = gjr_t_fit.params['gamma[1]']

In [26]:
val = {}

val["a_{t-1}"] = gjr_t_fit.resid[-1]
val["σ_{t-1}"] = gjr_t_fit.conditional_volatility[-1]
val["I_{t-1}"] = 1 if (gjr_t_fit.resid[-1] < 0) else 0

In [28]:
# First prediction 
a0 + (a1 + g1 * val["I_{t-1}"]) * (val["a_{t-1}"] ** 2) + (b1) * (val["σ_{t-1}"] ** 2)

1.4026563997768848

In [31]:
# Forecasting variance when the shock and variance is not known
forecasted_variance = 1.43262871
a0 + (a1 + b1 + g1/2) * forecasted_variance

1.4474348617105408

In [23]:
f_gjr_t[:10].flatten()

array([1.4026564 , 1.41770288, 1.43262871, 1.44743486, 1.46212229,
       1.47669196, 1.4911448 , 1.50548176, 1.51970376, 1.53381172])

# AR(1)-EGARCH(1, 0)

$$\log(\sigma_{t+1}^2) = 0.0109 + 0.1108 a_{t-1} + 0.9909\log(\sigma_{t}^2) -0.0362(|a_{t-1}| - \mathbb{E}[|a_{t-1}|]$$

or

$$\sigma_{t+1}^2 = \exp\big[0.015335 + 0.98749\, \log(\sigma_{t}^2)\big]$$

In [25]:
# GJR-GARCH(1,1)
a0 = egarch_n_fit.params['omega']
a1 = egarch_n_fit.params['alpha[1]']
b1 = egarch_n_fit.params['beta[1]']
g1 = egarch_n_fit.params['gamma[1]']

In [26]:
val = {}

val["a_{t-1}"] = egarch_n_fit.resid[-1]
val["σ_{t-1}"] = egarch_n_fit.conditional_volatility[-1]

In [27]:
val["log(σ_{t}^2)"] = a0 + a1 * val["a_{t-1}"] + b1 * np.log(val["σ_{t-1}"] ** 2) + g1 * (abs(val["a_{t-1}"]) - np.sqrt(2/np.pi))
np.exp(val["log(σ_{t}^2)"])

1.4475554594545732

$$\mathbb{E}[|a|] = \sqrt{\frac{2}{\pi}}$$
if $|a|\sim\mathcal{N}(0, 1)$

In [20]:
f_egarch_t = egarch_t_fit.forecast(horizon=1, align='origin', reindex=False).variance.values.T
f_egarch_t

array([[1.22814162]])